In [ ]:
from clickhouse_connect import get_client
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
from functools import lru_cache
import time

# 连接ClickHouse
client = get_client(
    host="xx",
    port=x,
    username="xx",
    password="xxxx"
)

# 配置参数
MAX_WORKERS = 24  # 根据你的服务器32线程，用24个
BATCH_SIZE = 100  # 每批处理100只股票

# 1. 获取撤单数据（全市场一次查询）
@lru_cache(maxsize=10)
def get_all_canceled_orders(date: str = '2025-03-03'):
    """获取全市场撤单ID"""
    sql = f"""
    SELECT code, bidno, askno
    FROM stock_base.zb
    WHERE date='{date}'
        AND date_time BETWEEN '{date} 09:15:00' AND '{date} 09:25:00'
        AND trade_flag='4'
        AND exg='1'
    """
    
    cancel_df = client.query_df(sql)
    cancel_dict = {}
    
    for _, row in cancel_df.iterrows():
        code = row['code']
        if code not in cancel_dict:
            cancel_dict[code] = set()
        
        if pd.notna(row['bidno']) and row['bidno'] != '':
            cancel_dict[code].add(int(row['bidno']))
        if pd.notna(row['askno']) and row['askno'] != '':
            cancel_dict[code].add(int(row['askno']))
    
    print(f"撤单数据: {len(cancel_dict)}只股票")
    return cancel_dict

# 2. 批量获取股票订单
def get_orders_batch(codes: list, date: str):
    """批量查询订单数据"""
    if not codes:
        return {}
    
    codes_str = "','".join(codes)
    sql = f"""
    SELECT code, price, volume, side, ordno
    FROM stock_base.zb
    WHERE code IN ('{codes_str}')
        AND date='{date}'
        AND date_time BETWEEN '{date} 09:15:00' AND '{date} 09:25:00'
        AND trade_flag NOT IN ('4', 'F')
        AND exg='1'
    """
    
    df = client.query_df(sql)
    orders_dict = {}
    
    if not df.empty:
        for code in codes:
            orders_dict[code] = df[df['code'] == code].copy()
    
    return orders_dict

# 3. 获取真实开盘价（批量）
def get_real_prices_batch(codes: list, date: str):
    """批量获取真实开盘价"""
    if not codes:
        return {}
    
    codes_str = "','".join(codes)
    sql = f"""
    SELECT code, open, pclose
    FROM stock_base.daily
    WHERE code IN ('{codes_str}')
        AND date='{date}'
        AND exg='1'
    """
    
    df = client.query_df(sql)
    price_dict = {}
    
    for _, row in df.iterrows():
        price_dict[row['code']] = (float(row['open']), float(row['pclose']))
    
    return price_dict

# 4. 优化的撮合算法
def calculate_open_price_fast(bid_list, ask_list, prev_close=None):
    """快速计算开盘价"""
    if bid_list.empty or ask_list.empty:
        return None, 0
    
    # 使用numpy数组
    bid_prices = bid_list['price'].values
    bid_volumes = bid_list['volume'].values
    ask_prices = ask_list['price'].values
    ask_volumes = ask_list['volume'].values
    
    # 所有价格
    all_prices = np.unique(np.concatenate([bid_prices, ask_prices]))
    all_prices.sort()
    
    best_price = None
    best_volume = 0
    best_diff = float('inf')
    
    for price in all_prices:
        # 买方累计
        bid_volume = bid_volumes[bid_prices >= price].sum()
        # 卖方累计
        ask_volume = ask_volumes[ask_prices <= price].sum()
        
        volume = min(bid_volume, ask_volume)
        if volume == 0:
            continue
        
        # 规则三检查
        bid_at_price = bid_volumes[bid_prices == price].sum()
        ask_at_price = ask_volumes[ask_prices == price].sum()
        
        if bid_at_price > 0 and ask_at_price > 0:
            if not (min(bid_at_price, volume) == bid_at_price or 
                   min(ask_at_price, volume) == ask_at_price):
                continue
        
        volume_diff = abs(bid_volume - ask_volume)
        
        # 更新最佳
        if volume > best_volume:
            best_price = price
            best_volume = volume
            best_diff = volume_diff
        elif volume == best_volume:
            if volume_diff < best_diff:
                best_price = price
                best_diff = volume_diff
            elif volume_diff == best_diff and prev_close:
                if abs(price - prev_close) < abs(best_price - prev_close):
                    best_price = price
    
    return best_price, best_volume

# 5. 处理单只股票
def process_stock(code, orders_df, cancel_dict, price_dict):
    """处理单只股票"""
    try:
        if orders_df.empty:
            return code, None, None, None, "无数据"
        
        # 过滤撤单
        if code in cancel_dict:
            mask = ~orders_df['ordno'].isin(cancel_dict[code])
            orders_df = orders_df[mask]
        
        # 分离买卖盘
        bid_df = orders_df[orders_df['side'] == b'B']
        ask_df = orders_df[orders_df['side'] == b'S']
        
        if bid_df.empty or ask_df.empty:
            return code, None, None, None, "买卖盘不全"
        
        # 排序
        bid_df = bid_df.sort_values(['price', 'ordno'], ascending=[False, True])
        ask_df = ask_df.sort_values(['price', 'ordno'], ascending=[True, True])
        
        # 获取真实价格
        prev_close = None
        real_open = None
        if code in price_dict:
            real_open, prev_close = price_dict[code]
        
        # 计算
        calc_open, volume = calculate_open_price_fast(bid_df, ask_df, prev_close)
        
        return code, calc_open, real_open, volume, None
    except Exception as e:
        return code, None, None, None, str(e)

# 6. 主函数
def main_simple(date='2025-03-03'):
    """简化版主程序"""
    start_time = time.time()
    
    print(f"开始处理 {date} 的数据...")
    
    # 获取所有股票
    sql = f"""
    SELECT DISTINCT code
    FROM stock_base.daily
    WHERE date='{date}' AND exg='1'
    ORDER BY toInt32(code)
    """
    
    codes_df = client.query_df(sql)
    all_codes = codes_df['code'].tolist()
    print(f"共 {len(all_codes)} 只股票")
    
    # 获取基础数据
    cancel_dict = get_all_canceled_orders(date)
    
    # 分批处理
    results = []
    error_no_data = []
    error_no_price = []
    
    for i in range(0, len(all_codes), BATCH_SIZE):
        batch_codes = all_codes[i:i+BATCH_SIZE]
        print(f"处理批次 {i//BATCH_SIZE+1}/{(len(all_codes)+BATCH_SIZE-1)//BATCH_SIZE}")
        
        # 批量获取数据
        orders_dict = get_orders_batch(batch_codes, date)
        price_dict = get_real_prices_batch(batch_codes, date)
        
        # 并行处理
        batch_results = []
        with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            futures = []
            for code in batch_codes:
                orders_df = orders_dict.get(code, pd.DataFrame())
                future = executor.submit(process_stock, code, orders_df, cancel_dict, price_dict)
                futures.append(future)
            
            for future in as_completed(futures):
                batch_results.append(future.result())
        
        # 统计结果
        for code, calc_open, real_open, volume, error in batch_results:
            if error:
                error_no_data.append(code)
                continue
            
            if real_open is None:
                error_no_price.append(code)
                continue
            
            is_correct = calc_open is not None and abs(calc_open - real_open) < 0.001
            results.append({
                'code': code,
                'calc_open': calc_open,
                'real_open': real_open,
                'volume': volume,
                'is_correct': is_correct,
                'error': calc_open - real_open if calc_open else None
            })
    
    # 输出统计
    total_time = time.time() - start_time
    correct_count = sum(1 for r in results if r['is_correct'])
    
    print(f"\n{'='*50}")
    print(f"处理完成!")
    print(f"总耗时: {total_time:.2f}秒")
    print(f"总股票: {len(all_codes)}")
    print(f"计算成功: {len(results)}")
    print(f"计算正确: {correct_count}")
    print(f"数据异常: {len(error_no_data)}")
    print(f"价格缺失: {len(error_no_price)}")
    print(f"正确率: {correct_count/max(1, len(results))*100:.1f}%")
    print(f"平均每只: {total_time/len(all_codes)*1000:.1f}ms")
    
    # 保存结果
    if results:
        results_df = pd.DataFrame(results)
        results_df.to_csv(f'开盘价结果_{date}1.csv', index=False, encoding='utf-8-sig')
        print(f"结果已保存到: 开盘价结果_{date}1.csv")
    
    return results

# 运行
if __name__ == "__main__":
    main_simple('2025-03-03')

开始处理 2025-03-03 的数据...
共 2850 只股票
撤单数据: 2634只股票
处理批次 1/29
处理批次 2/29
处理批次 3/29
处理批次 4/29
处理批次 5/29
处理批次 6/29
处理批次 7/29
处理批次 8/29
处理批次 9/29
处理批次 10/29
处理批次 11/29
处理批次 12/29
处理批次 13/29
处理批次 14/29
处理批次 15/29
处理批次 16/29
处理批次 17/29
处理批次 18/29
处理批次 19/29
处理批次 20/29
处理批次 21/29
处理批次 22/29
处理批次 23/29
处理批次 24/29
处理批次 25/29
处理批次 26/29
处理批次 27/29
处理批次 28/29
处理批次 29/29

处理完成!
总耗时: 36.98秒
总股票: 2850
计算成功: 2850
计算正确: 2703
数据异常: 0
价格缺失: 0
正确率: 94.8%
平均每只: 13.0ms
结果已保存到: 开盘价结果_2025-03-031.csv
